# 기본 머신러닝 모델 학습

이 노트북은 scikit-learn을 사용하여 간단한 분류 모델을 학습하는 예제입니다.

## 학습 내용
- 데이터 로드 및 전처리
- 모델 학습
- 모델 평가
- 모델 저장

In [ ]:
# 필요한 라이브러리 설치
!pip install -q scikit-learn pandas numpy matplotlib seaborn

In [ ]:
# 라이브러리 임포트
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris, load_wine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import joblib
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 설정 (Colab용)
plt.rcParams['font.family'] = 'DejaVu Sans'

## 1. 데이터 로드

In [ ]:
# Iris 데이터셋 로드
iris = load_iris()
X = iris.data
y = iris.target

# 데이터프레임으로 변환
df = pd.DataFrame(X, columns=iris.feature_names)
df['target'] = y

print(f"데이터 형태: {df.shape}")
print(f"\n특성 정보:")
print(df.info())
print(f"\n처음 5개 행:")
df.head()

## 2. 데이터 탐색

In [ ]:
# 기술 통계
print("기술 통계:")
df.describe()

In [ ]:
# 타겟 분포
plt.figure(figsize=(8, 5))
df['target'].value_counts().plot(kind='bar')
plt.title('Target Distribution')
plt.xlabel('Class')
plt.ylabel('Count')
plt.xticks(rotation=0)
plt.show()

## 3. 데이터 전처리

In [ ]:
# 학습/테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"학습 데이터 크기: {X_train.shape}")
print(f"테스트 데이터 크기: {X_test.shape}")

In [ ]:
# 특성 스케일링
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("스케일링 완료")
print(f"평균: {X_train_scaled.mean(axis=0)}")
print(f"표준편차: {X_train_scaled.std(axis=0)}")

## 4. 모델 학습

In [ ]:
# Random Forest 모델 생성 및 학습
model = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    random_state=42,
    n_jobs=-1
)

print("모델 학습 중...")
model.fit(X_train_scaled, y_train)
print("모델 학습 완료!")

## 5. 모델 평가

In [ ]:
# 예측
y_train_pred = model.predict(X_train_scaled)
y_test_pred = model.predict(X_test_scaled)

# 정확도
train_acc = accuracy_score(y_train, y_train_pred)
test_acc = accuracy_score(y_test, y_test_pred)

print(f"학습 정확도: {train_acc:.4f}")
print(f"테스트 정확도: {test_acc:.4f}")

In [ ]:
# 분류 리포트
print("분류 리포트:")
print(classification_report(y_test, y_test_pred, target_names=iris.target_names))

In [ ]:
# 혼동 행렬
cm = confusion_matrix(y_test, y_test_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=iris.target_names, 
            yticklabels=iris.target_names)
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

In [ ]:
# 특성 중요도
feature_importance = pd.DataFrame({
    'feature': iris.feature_names,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)

plt.figure(figsize=(10, 6))
plt.barh(feature_importance['feature'], feature_importance['importance'])
plt.xlabel('Importance')
plt.title('Feature Importance')
plt.gca().invert_yaxis()
plt.show()

print("\n특성 중요도:")
print(feature_importance)

## 6. 모델 저장

In [ ]:
# 모델 및 스케일러 저장
joblib.dump(model, 'iris_model.pkl')
joblib.dump(scaler, 'scaler.pkl')

print("모델과 스케일러가 저장되었습니다.")
print("- iris_model.pkl")
print("- scaler.pkl")

## 7. 저장된 모델 로드 및 예측

In [ ]:
# 저장된 모델 로드
loaded_model = joblib.load('iris_model.pkl')
loaded_scaler = joblib.load('scaler.pkl')

# 새로운 데이터로 예측
sample_data = X_test[:5]
sample_scaled = loaded_scaler.transform(sample_data)
predictions = loaded_model.predict(sample_scaled)

print("샘플 예측:")
for i, (pred, true) in enumerate(zip(predictions, y_test[:5])):
    print(f"샘플 {i+1}: 예측={iris.target_names[pred]}, 실제={iris.target_names[true]}")

## 요약

이 노트북에서는 다음을 수행했습니다:
1. ✅ Iris 데이터셋 로드 및 탐색
2. ✅ 데이터 전처리 (분할 및 스케일링)
3. ✅ Random Forest 모델 학습
4. ✅ 모델 평가 (정확도, 분류 리포트, 혼동 행렬)
5. ✅ 특성 중요도 분석
6. ✅ 모델 저장 및 로드

다음 단계: 실험 추적 및 버전 관리를 위한 MLflow 사용